## Training our Pet Classifier

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras

Using TensorFlow backend.


In [2]:
num_classes = 2
img_rows,img_cols = 150,150
batch_size = 100

In [3]:
train_data_dir = '/users/dipit/Documents/MLDL/CNN/datasets/training_set'
validation_data_dir = '/users/dipit/Documents/MLDL/CNN/datasets/test_set'

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 0.3,
                                   width_shift_range = 0.3,
                                   height_shift_range = 0.3,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest')

In [5]:
train_generator = train_data_gen.flow_from_directory(directory = train_data_dir,
                                                    target_size = (img_rows,img_cols),
                                                    batch_size = batch_size,
                                                    class_mode = 'binary',
                                                    shuffle = True)

Found 8000 images belonging to 2 classes.


In [6]:
validation_data_gen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_data_gen.flow_from_directory(directory = validation_data_dir,
                                                              target_size = (img_rows,img_cols),
                                                              batch_size = batch_size,
                                                              class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Defining the Model

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout

def convnet():
    model = Sequential([
    Conv2D(32,(3,3),padding = 'same',activation = 'relu',input_shape = (img_rows,img_cols,3)),
    Conv2D(32,(3,3),activation = 'relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
        
        
    Conv2D(64,(3,3),activation = 'relu',padding = 'same'),
    Conv2D(64,(3,3),activation = 'relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
        
    Flatten(),
    Dense(128,activation = 'relu'),
    Dropout(0.5),
    Dense(2,activation = 'sigmoid'),        
    ])
    return model


model = convnet()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 148, 148, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 74, 74, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)       

In [8]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
checkpoint = ModelCheckpoint("pet_classifier_model.h5",
                            monitor = 'val_accuracy',
                            mode = 'min',
                            save_best_only = True,
                            verbose = 1)

earlystop = EarlyStopping(monitor = 'val_accuracy',
                         min_delta = 0,
                         patience = 3,
                         verbose =1,
                         restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy',
                             factor = 0.2,
                             patience = 3,
                             verbose = 1,
                             min_delta = 0.0001)
callbacks = [earlystop,checkpoint,reduce_lr]

In [9]:
model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [10]:
num_train_samples = 8000
num_val_samples = 2000

In [11]:
history = model.fit_generator(train_generator,
                              epochs = 5,
                              steps_per_epoch = (num_train_samples/batch_size),
                              validation_data = validation_generator,
                              validation_steps = (num_val_samples/batch_size),
                              callbacks = callbacks)

Epoch 1/5
80/80 [==============================] - 572s 7s/step - loss: 0.7647 - accuracy: 0.4956 - val_loss: 0.6932 - val_accuracy: 0.5000

Epoch 00001: val_accuracy improved from inf to 0.50000, saving model to pet_classifier_model.h5
Epoch 2/5
80/80 [==============================] - 565s 7s/step - loss: 0.6926 - accuracy: 0.5190 - val_loss: 0.6915 - val_accuracy: 0.5815

Epoch 00002: val_accuracy did not improve from 0.50000
Epoch 3/5
80/80 [==============================] - 565s 7s/step - loss: 0.6908 - accuracy: 0.5196 - val_loss: 0.6929 - val_accuracy: 0.5110

Epoch 00003: val_accuracy did not improve from 0.50000
Epoch 4/5
80/80 [==============================] - 569s 7s/step - loss: 0.6912 - accuracy: 0.5271 - val_loss: 0.6747 - val_accuracy: 0.5710

Epoch 00004: val_accuracy did not improve from 0.50000
Epoch 5/5
80/80 [==============================] - 562s 7s/step - loss: 0.6931 - accuracy: 0.5154 - val_loss: 0.6855 - val_accuracy: 0.5000
Restoring model weights from the en

In [21]:
scores = model.evaluate_generator(validation_generator,steps = num_val_samples // batch_size,
                                 verbose = 1)
print("\n Test Result: %3.f loss:%3.f" %(scores[1]*100 , scores[0]))

20/20 [==============================] - 32s 2s/step

 Test Result:  58 loss:  1


In [22]:
model.save('pet_classifier_model.h5')

In [23]:
y_pred = model.predict(validation_generator, steps = num_val_samples // batch_size)

In [24]:
y_pred_label = np.argmax(y_pred, axis=1)

In [25]:
y_pred_label

array([0, 0, 0, ..., 0, 0, 0])

## Confusion Matrix

In [26]:
from sklearn.metrics import confusion_matrix,classification_report
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes,y_pred_label))
print('Classification Report')
class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
print(classification_report(validation_generator.classes,y_pred_label,target_names = classes))

Confusion Matrix
[[864 136]
 [851 149]]
Classification Report
              precision    recall  f1-score   support

        cats       0.50      0.86      0.64      1000
        dogs       0.52      0.15      0.23      1000

    accuracy                           0.51      2000
   macro avg       0.51      0.51      0.43      2000
weighted avg       0.51      0.51      0.43      2000



In [1]:
from keras.preprocessing import image
import os
import cv2
from os import listdir
from os.path import isfile,join
import re

def draw_test(name,pred,im,true_label):
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im,160,0,0,500,cv2.BORDER_CONSTANT,value = BLACK)
    cv2.putText(expanded_image,"predicted - "+pred,(20,60),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2.putText(expanded_image,"true - "+true_label,(20,120),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow(name,expanded_image)
    
def getRandomImage(path, img_width, img_height):
    folders = list(filter(lambda x :os.path.isdir(os.path.join(path,x)),os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    file_path = path + "/" + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    final_path = file_path + "/" + image_name
    return image.load_img(final_path, target_size = (img_width,img_height)),final_path,path_class

img_width,img_height = 150,150

files = []
predictions = []
true_labels = []

for i in range(0,10):
    path = '/users/dipit/Documents/MLDL/CNN/test_pictures_pets'
    img,final_path,true_label = getRandomImage(path,img_width,img_height)
    files.append(final_path)
    true_labels.append(true_label)
    x = image.img_to_array(img)
    x = x*1./255
    x = np.expand_dims(x,axis=0)
    images = np.vstack([x])
    classes = model.predict_classes(images,batch_size = 2)
    predictions.append(classes)
    
for i in range(0,len(files)):
    image = cv2.imread((files[i]))
    draw_test("Prediction",class_labels[predictions[i][0]],image,true_labels[i])
    cv2.waitKey(0)
    
cv2.destoryAllWindows()

Using TensorFlow backend.


NameError: name 'np' is not defined